1. selenium으로 토스 GoolePlay에 접속
2. 전체리뷰 보기 클릭
3. 리뷰창이 뜨면 최신순으로 정렬
4. 마우스 스크롤을 해서 오늘부터 2년전 날짜가 나올 때까지 스크롤
5. 데이터 수집
6. csv파일로 저장 → DB 저장
7. 수집코드 함수화
8. KB, 신한, 하나, 뱅크샐러드, 카카오뱅크 리뷰도 저장

In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from general_dbio import *
print(dir())

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'create_engine', 'datetime', 'exit', 'get_ipython', 'open', 'pd', 'pymysql', 'quit', 'relativedelta', 'saveDB', 'time', 'timedelta']


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [3]:
end_date = datetime.today() - relativedelta(years=2) - timedelta(days=1)
end_date_text = end_date.strftime('%Y년 %m월 %d일')
print(end_date)
print(end_date_text)
print( datetime.today() >= datetime.strptime(end_date_text, '%Y년 %m월 %d일'))

2023-04-13 23:08:29.909200
2023년 04월 13일
True


In [23]:
# shinhan_sol='com.shinhan.sbanking'
# toss='viva.republica.toss'
bank_list = dict(kb_star='com.kbstar.kbbank', hana='com.kebhana.hanapush', banksalad='com.rainist.banksalad2', kakaobank='com.kakaobank.channel')
bank_list

{'kb_star': 'com.kbstar.kbbank',
 'hana': 'com.kebhana.hanapush',
 'banksalad': 'com.rainist.banksalad2',
 'kakaobank': 'com.kakaobank.channel'}

In [20]:
from selenium.common.exceptions import NoSuchElementException

def safe_find_text(idx, bank_name, element_name, selector, by=By.CSS_SELECTOR, driver=None):
    try:
#         print(f'idx 입니다 : {idx}')
#         print(f'bank name 입니다 : {bank_name}')
#         print(f'element name 입니다 : {element_name}')
#         print(f'selector 출력해보는중!!! {selector}')
#         print(f'return값 출력해보는 중!!! {driver.find_element(by, selector).text}')
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((by, selector))
        )
        return element.text
    except NoSuchElementException:
        print(f'{bank_name}에서 {idx} : {element_name} 수집 중')
        return None
    except Exception as e:
        print(f'{bank_name}에서 {element_name} 수집 중')
        print(f"⚠️ 예외 발생: {e} | selector: {selector}")
        return None

def safe_find_attr(idx, bank_name, element_name, selector, attr, by=By.CSS_SELECTOR, driver=None):
    try:
        return float(driver.find_element(by, selector).get_attribute(attr)[-10])
    except NoSuchElementException:
        return None
    except Exception as e:
        print(f'{bank_name}에서 {element_name} 수집 중')
        print(f"⚠️ 예외 발생: {e} | selector: {selector} | attr: {attr}")
        return None


In [14]:
driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
options=options
)

driver.get(f'https://play.google.com/store/apps/details?id=viva.republica.toss')

reviews_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label*='평점 및 리뷰 자세히 알아보기']"))
        )
reviews_btn.click()
order_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1"))
    )
order_btn.click()
recent_order = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))
    )
recent_order.click()

reviews_modal = driver.find_element(By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd')

In [9]:
driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type(1) header > div div.X5PpBb').text

'유7080'

In [ ]:
columns = ['날짜', '닉네임','평점','리뷰내용']

for bank, domain in bank_list.items():
    idx = 1
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

    driver.get(f'https://play.google.com/store/apps/details?id={domain}')

    reviews_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label*='평점 및 리뷰 자세히 알아보기']"))
        )
    reviews_btn.click()
    order_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1"))
        )
    order_btn.click()
    recent_order = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))
        )
    time.sleep(1)
    recent_order.click()


    reviews_modal = driver.find_element(By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd')

    try: 
        while True:
            time.sleep(1)
            date = safe_find_text(idx, bank, 'date', f'div.RHo1pe:nth-of-type({idx}) header span.bp9Aid', driver=driver)
            nickname = safe_find_text(idx, bank, 'nickname', f'div.RHo1pe:nth-of-type({idx}) header > div div.X5PpBb', driver=driver)
            rating = safe_find_attr(idx, bank, 'rating', f'div.RHo1pe:nth-of-type({idx}) header .Jx4nYe > div', 'aria-label', driver=driver)
            content = safe_find_text(idx, bank, 'content', f'div.RHo1pe:nth-of-type({idx}) div.h3YV2d', driver=driver)

            print(idx, date, nickname, rating, content, end='\r')
            
            if not date:
                continue

            date_cast = datetime.strptime(date, '%Y년 %m월 %d일')
            if date_cast <= end_date:
                break

            temp = (date, nickname, rating, content)
            df = pd.DataFrame([temp], columns=columns)
            saveDB(df, 'google_play_reviews', bank)
            time.sleep(1)
            try:
                driver.execute_script("arguments[0].scrollTop += 1000;", reviews_modal)  # 아래로 500px씩 스크롤
                time.sleep(1)
            except:
                print('스크롤 안됨')

            idx += 1
    except Exception as e:
        print(f'예외 발생: {type(e).__name__} - {e}')
    finally:
        driver.quit()

In [12]:
driver.quit()